In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [3]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [19]:
train_gen = SequenceGenSpec(train)
#train_gen = mixupWrapper(SequenceGenSpec(train),4e-2) # does not improve
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [12]:
convNets = Sequential(Dropout(0.5),
                      DenseBlock(129),
                      Dropout(0.5),
                      DenseBlock(258),
                      Dropout(0.5),
                      GRU_NCL(516,516,2,batch_first=True,dropout=0.5,bidirectional=True,returnH=True))

In [13]:
linear = Sequential(Dropout(0.5),Linear(516*4,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [14]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [15]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [16]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [10]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.7350632674418964, val_loss:2.621134042739868
epoch:1, train_loss:2.4258977074462633, val_loss:2.4757230281829834
epoch:2, train_loss:2.269232344742005, val_loss:2.4561374187469482
epoch:3, train_loss:2.2503889243190107, val_loss:2.553551435470581
epoch:4, train_loss:2.216916866027392, val_loss:2.4548556804656982
Training completed in 86.91920208930969s


Submission

In [11]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [12]:
test_gen = SequenceGenSpecTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [13]:
submission.iloc[:,1] = predict(model,test_gen)

In [14]:
submission.to_csv('../Submission/sub_spec_2d.csv',index=False)